# LangChain 实战：房产销售聊天机器人

## 使用 GPT-4 构造销售话术数据

ChatGPT 分享链接：https://chat.openai.com/share/f3e4b9b0-95fb-4c6a-a3c7-f901dd194c91


使用 ChatGPT 构造销售数据的 Prompt 示例：

```
你是中国顶级的房地产销售，现在培训职场新人，请给出100条实用的销售话术。

每条销售话术以如下格式给出：
[客户问题]
[销售回答]

```

GPT-4 回复结果：

```
在房地产销售行业中，精心准备的销售话术可以极大地提高成交率。以下是一些适用于中国房地产销售的实用话术：

1.  
[客户问题] 这个小区交通便利吗？
[销售回答] 当然了，这个小区距离地铁站只有几分钟的步行距离，而且附近有多条公交线路，非常方便。

2.  
[客户问题] 我担心楼下太吵。
[销售回答] 这个小区特别注重居住体验，我们有良好的隔音设计，并且小区内部规划了绿化区域，可以有效降低噪音。

3.  
[客户问题] 我看房价还在涨，这个投资回报怎么样？
[销售回答] 这个区域未来有大量的商业和基础设施建设，所以从长期来看，投资回报非常有保证。

4.  
[客户问题] 有没有学校？
[销售回答] 附近有多所优质的学校，非常适合有孩子的家庭。

5.  
[客户问题] 物业管理怎么样？
[销售回答] 我们的物业管理得到了业主一致好评，服务非常到位。

6.  
[客户问题] 我想要南向的房子。
[销售回答] 很好，我们确实有一些朝南的单位，它们的采光特别好。

7.  
[客户问题] 这个小区安全吗？
[销售回答] 当然，我们24小时安保巡逻，还有先进的监控系统。

8.  
[客户问题] 预计什么时候交房？
[销售回答] 根据目前的进度，我们预计将在明年底交房。

9.  
[客户问题] 我不想要一楼的房子。
[销售回答] 我理解您的顾虑，我们还有多个楼层的房源可以选择。

10.  
[客户问题] 有优惠吗？
[销售回答] 当然，如果您现在下订，我们可以给您一些优惠。
```


## 使用 Document Transformers 模块来处理原始数据


将 ChatGPT 生成的结果保存到 [real_estate_sales_data.txt](real_estate_sales_data.txt) 文件中

In [1]:
with open("real_estate_sales_data.txt",'r', encoding='utf-8') as f:
    real_estate_sales = f.read()

### 使用 CharacterTextSplitter 来进行文本分割

- 基于单字符来进行文本分割（separator）
- 基于字符数来决定文本块长度（chunk_size）

参考示例：

```python
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(        
    separator = "\n\n",
    chunk_size = 1000,
    chunk_overlap  = 200,
    length_function = len,
    is_separator_regex = False,
)
```


In [2]:
from langchain.text_splitter import CharacterTextSplitter

In [3]:
text_splitter = CharacterTextSplitter(        
    separator = r'\d+\.',
    chunk_size = 100,
    chunk_overlap  = 0,
    length_function = len,
    is_separator_regex = True,
)

In [4]:
docs = text_splitter.create_documents([real_estate_sales])

In [5]:
docs[0]

Document(page_content='[客户问题] 这个小区交通便利吗？\n[销售回答] 当然了，这个小区距离地铁站只有几分钟的步行距离，而且附近有多条公交线路，非常方便。')

In [6]:
len(docs)

70

### 使用 Faiss 作为向量数据库，持久化存储房产销售 问答对（QA-Pair）

In [8]:
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS

db = FAISS.from_documents(docs, OpenAIEmbeddings(api_key="sk-MLOkqReWfMXcorYNEfA4321bA32a4c6e87FbA00839533a0a", base_url="https://api.xiaoai.plus/v1"))

KeyboardInterrupt: 

In [ ]:
query = "小区吵不吵"

In [ ]:
answer_list = db.similarity_search(query)

In [ ]:
for ans in answer_list:
    print(ans.page_content + "\n")

[客户问题] 我担心楼下太吵。
[销售回答] 这个小区特别注重居住体验，我们有良好的隔音设计，并且小区内部规划了绿化区域，可以有效降低噪音。

[客户问题] 我担心楼下的商业活动会很吵。
[销售回答] 我们在规划时就已经考虑到这一点，商业区和居住区有一定的距离和隔音设计。

[客户问题] 我喜欢安静，这里噪音大吗？
[销售回答] 我们特意进行了隔音设计，并且小区内部也有绿化带，整体非常安静。

[客户问题] 我担心小区会很拥挤。
[销售回答] 这个小区总体规划非常合理，保证了每个单元之间有足够的空间。



In [ ]:
db.save_local("real_estates_sale")

### 使用 retriever 从向量数据库中获取结果

#### 使用参数 `k` 指定返回结果数量


In [ ]:
# 实例化一个 TopK Retriever
topK_retriever = db.as_retriever(search_kwargs={"k": 3})

In [ ]:
topK_retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001E6D3274910>, search_kwargs={'k': 3})

In [ ]:
docs = topK_retriever.invoke(query)
for doc in docs:
    print(doc.page_content + "\n")

[客户问题] 我担心楼下太吵。
[销售回答] 这个小区特别注重居住体验，我们有良好的隔音设计，并且小区内部规划了绿化区域，可以有效降低噪音。

[客户问题] 我担心楼下的商业活动会很吵。
[销售回答] 我们在规划时就已经考虑到这一点，商业区和居住区有一定的距离和隔音设计。

[客户问题] 我喜欢安静，这里噪音大吗？
[销售回答] 我们特意进行了隔音设计，并且小区内部也有绿化带，整体非常安静。



In [ ]:
docs = topK_retriever.get_relevant_documents("你们有没有1000万的豪宅啊？")

c:\ProgramData\miniconda3\envs\langchain\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


In [ ]:
for doc in docs:
    print(doc.page_content + "\n")

[客户问题] 你们会提供家具吗？
[销售回答] 我们的精装房会提供基础家具和家电，让您拎包入住。

[客户问题] 都有哪些户型？
[销售回答] 我们有从一室到四室不等的多种户型，定能满足您不同的居住需求。

[客户问题] 我不想要一楼的房子。
[销售回答] 我理解您的顾虑，我们还有多个楼层的房源可以选择。



#### 使用 similarity_score_threshold 设置阈值，提升结果的相关性质量

In [ ]:
# 实例化一个 similarity_score_threshold Retriever
retriever = db.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"score_threshold": 0.8}
)

In [ ]:
docs = retriever.get_relevant_documents(query)
for doc in docs:
    print(doc.page_content + "\n")

[客户问题] 我担心楼下太吵。
[销售回答] 这个小区特别注重居住体验，我们有良好的隔音设计，并且小区内部规划了绿化区域，可以有效降低噪音。



### 提取向量数据库中的`销售回答`

In [ ]:
docs = retriever.get_relevant_documents(query)

In [ ]:
docs[0].page_content

'[客户问题] 我担心楼下太吵。\n[销售回答] 这个小区特别注重居住体验，我们有良好的隔音设计，并且小区内部规划了绿化区域，可以有效降低噪音。'

In [ ]:
docs[0].page_content.split("[销售回答] ")

['[客户问题] 我担心楼下太吵。\n', '这个小区特别注重居住体验，我们有良好的隔音设计，并且小区内部规划了绿化区域，可以有效降低噪音。']

In [ ]:
ans = docs[0].page_content.split("[销售回答] ")[-1]

In [ ]:
ans

'这个小区特别注重居住体验，我们有良好的隔音设计，并且小区内部规划了绿化区域，可以有效降低噪音。'

#### 尝试各种问题

In [ ]:
from typing import List

def sales(query: str, score_threshold: float=0.8) -> List[str]:
    retriever = db.as_retriever(search_type="similarity_score_threshold", search_kwargs={"score_threshold": score_threshold})    
    docs = retriever.get_relevant_documents(query)
    ans_list = [doc.page_content.split("[销售回答] ")[-1] for doc in docs]

    return ans_list

In [ ]:
query = "我想离医院近点"

print(sales(query))

[]


c:\ProgramData\miniconda3\envs\langchain\lib\site-packages\langchain_core\vectorstores\base.py:796: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


In [ ]:
print(sales(query, 0.75))

['有的，距离我们小区不远就有几家大型综合医院。', '是的，附近有多家大型医院，医疗资源非常丰富。']


In [ ]:
query = "价格200万以内"

print(f"score:0.8 ans: {sales(query)}\n")
print(f"score:0.75 ans: {sales(query, 0.75)}\n")
print(f"score:0.5 ans: {sales(query, 0.5)}\n")

c:\ProgramData\miniconda3\envs\langchain\lib\site-packages\langchain_core\vectorstores\base.py:796: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


score:0.8 ans: []



c:\ProgramData\miniconda3\envs\langchain\lib\site-packages\langchain_core\vectorstores\base.py:796: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.75
  warnings.warn(


score:0.75 ans: []

score:0.5 ans: ['我们有不同户型和付款方案，一定有适合您预算的。', '我们的房子位于黄金地段，升值潜力很大，转卖起来不会有问题。', '定金金额取决于您选择的房型和付款方式，我们可以详细为您解释。', '这个区域正在快速发展，未来的升值潜力非常大。']



#### 当向量数据库中没有合适答案时，使用大语言模型能力

In [ ]:
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4-1106-preview", temperature=0.5, api_key="sk-MLOkqReWfMXcorYNEfA4321bA32a4c6e87FbA00839533a0a", base_url="https://api.xiaoai.plus/v1")
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=db.as_retriever(search_type="similarity_score_threshold",
                                                                 search_kwargs={"score_threshold": 0.8}))

In [ ]:
qa_chain({"query": "你们小区有200万的房子吗？"})

c:\ProgramData\miniconda3\envs\langchain\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
c:\ProgramData\miniconda3\envs\langchain\lib\site-packages\langchain_core\vectorstores\base.py:796: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


{'query': '你们小区有200万的房子吗？',
 'result': "I'm sorry, but as an AI, I don't have access to real estate listings or the ability to provide information about housing in specific areas. Please check with local real estate agents or online real estate platforms for information about housing prices in the area you're interested in."}

In [ ]:
qa_chain({"query": "小区吵不吵"})

{'query': '小区吵不吵',
 'result': '这个小区特别注重居住体验，有良好的隔音设计，并且小区内部规划了绿化区域，可以有效降低噪音。所以整体来说不会太吵。'}

In [ ]:
print(sales("小区吵不吵"))

['这个小区特别注重居住体验，我们有良好的隔音设计，并且小区内部规划了绿化区域，可以有效降低噪音。']


## 加载 FAISS 向量数据库已有结果

In [ ]:
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS

db = FAISS.load_local("real_estates_sale", OpenAIEmbeddings(api_key="sk-MLOkqReWfMXcorYNEfA4321bA32a4c6e87FbA00839533a0a", base_url="https://api.xiaoai.plus/v1"), allow_dangerous_deserialization=True)

In [ ]:
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4", temperature=0.5, api_key="sk-MLOkqReWfMXcorYNEfA4321bA32a4c6e87FbA00839533a0a", base_url="https://api.xiaoai.plus/v1")
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=db.as_retriever(search_type="similarity_score_threshold",
                                                                 search_kwargs={"score_threshold": 0.8}))

In [ ]:
qa_chain({"query": "我想买别墅，你们有么"})

c:\ProgramData\miniconda3\envs\langchain\lib\site-packages\langchain_core\vectorstores\base.py:796: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


{'query': '我想买别墅，你们有么',
 'result': '对不起，我是一个人工智能助手，我不能提供房地产服务。我建议你联系一个房地产代理或者搜索网上的房地产列表。'}

In [ ]:
# 输出内部 Chain 的日志
qa_chain.combine_documents_chain.verbose = True

In [ ]:
qa_chain({"query": "我想买别墅，你们有么"})

c:\ProgramData\miniconda3\envs\langchain\lib\site-packages\langchain_core\vectorstores\base.py:796: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(




> Entering new StuffDocumentsChain chain...

> Finished chain.


{'query': '我想买别墅，你们有么', 'result': '对不起，我不能帮助你购买别墅。我是一个人工智能，主要用于提供信息和回答问题。'}

In [ ]:
# 返回向量数据库的检索结果
qa_chain.return_source_documents = True

In [ ]:
result = qa_chain({"query": "我想买别墅，你们有么"})

c:\ProgramData\miniconda3\envs\langchain\lib\site-packages\langchain_core\vectorstores\base.py:796: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(




> Entering new StuffDocumentsChain chain...

> Finished chain.


In [ ]:
result

{'query': '我想买别墅，你们有么',
 'result': '对不起，我不能帮助您购买别墅。我是一个人工智能，我只能提供信息和回答问题。',
 'source_documents': []}